In [1]:
import pyiceberg
import pyiceberg.catalog

In [2]:
catalog = pyiceberg.catalog.load_catalog(                                                                                                                                                                                                                                    
            uri="http://nessie:19120/iceberg",                                                                                                                                                                                                                                            
            properties={                                                                                                                                                                                                                                                          
                "s3": {                                                                                                                                                                                                                                                           
                    "endpoint": "http://minio:9000",                                                                                                                                                                                                         
                    "access-key-id": "admin",                                                                                                                                                                                                         
                    "secret-access-key": "miniopass",                                                                                                                                                                                                 
                }                                                                                                                                                                                                                                                                 
            },                                                                                                                                                                                                                                                                    
        )  

In [ ]:
catalog.create_namespace("public")

In [32]:
from pyiceberg.schema import Schema
from pyiceberg.types import NestedField, StringType, DoubleType, IntegerType
from pyiceberg.partitioning import PartitionField, PartitionSpec
from pyiceberg.transforms import IdentityTransform
from pyiceberg.io.pyarrow import schema_to_pyarrow

In [36]:
schema = Schema(
    NestedField(1, "city", StringType(), required=False),
    NestedField(2, "lat", DoubleType(), required=False),
    NestedField(3, "long", DoubleType(), required=False),
    NestedField(4, "index", IntegerType(), required=True),
)

partition_spec = PartitionSpec(   
    PartitionField(source_id=4, field_id=1000, transform=IdentityTransform(), name="index_partition"),
)

tbl = catalog.create_table("public.cities", schema=schema, partition_spec=partition_spec)

In [26]:
catalog.drop_table("public.cities")

In [37]:
import pyarrow as pa

df = pa.Table.from_pylist(
    [
        {"city": "Amsterdam", "lat": 52.371807, "long": 4.896029, "index": 1},
        {"city": "San Francisco", "lat": 37.773972, "long": -122.431297, "index": 1},
        {"city": "Drachten", "lat": 53.11254, "long": 6.0989, "index": 2},
        {"city": "Paris", "lat": 48.864716, "long": 2.349014, "index": 2},
    ],
    schema=schema_to_pyarrow(tbl.schema())
)

In [44]:
df2 = pa.Table.from_pylist(
    [
        {"city": "Amsterdam", "lat": 52.371807, "long": 4.896029, "index": 1},
        {"city": "Null Island", "lat": 0.0, "long": 0.0, "index": 3},
    ],
    schema=schema_to_pyarrow(tbl.schema())
)

In [42]:
tbl.overwrite(df)

In [45]:
tbl.overwrite(df2)

In [46]:
tbl.scan().to_arrow()

pyarrow.Table
city: large_string
lat: double
long: double
index: int32 not null
----
city: [["Amsterdam"],["Null Island"]]
lat: [[52.371807],[0]]
long: [[4.896029],[0]]
index: [[1],[3]]